In [4]:
import pandas as pd
import numpy as np
import random
from rainbow_tqdm import tqdm
from tqdm.auto import tqdm
import gc
from implicit.evaluation import mean_average_precision_at_k,train_test_split
from implicit.approximate_als import FaissAlternatingLeastSquares
from implicit.nearest_neighbours import CosineRecommender, BM25Recommender,TFIDFRecommender
from implicit.cpu.lmf import LogisticMatrixFactorization
import scipy.sparse as sp
from implicit.gpu.bpr import BayesianPersonalizedRanking
from implicit.gpu.als import AlternatingLeastSquares

In [5]:
train_df = pd.read_csv('train.csv',parse_dates=['timestamp'])

C:\Users\kostopr4v\AppData\Local\Temp\ipykernel_2588\1248856124.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  train_df = pd.read_csv('train.csv',parse_dates=['timestamp'])


In [3]:
train_df

interaction_type      timestamp  user_id  video_id
0                   view  1696271079748   126492    228525
1                   view  1696271079760   117764    204343
2                   view  1696271079988    11347    201337
3                   view  1696271080451   125274    221842
4                   view  1696271080451   125274    221842
...                  ...            ...      ...       ...
5657319             view  1696946405821    20627     42064
5657320             view  1696946406884    73913    242899
5657321             view  1696946407292    53555    241941
5657322             view  1696946407461    97851    222069
5657323             like  1696946407541    63223    180394

[5657324 rows x 4 columns]

In [4]:
def chrono_split(
        df: pd.DataFrame, 
        split_by_column: str = 'user_id', 
        ratio: float = 0.7, 
        col_timestamp: str = 'timestamp'):

    df = df.sort_values([split_by_column, col_timestamp])
    groups = df.groupby(split_by_column)

    df["count"] = groups[split_by_column].transform("count")
    df["rank_s"] = groups.cumcount() + 1

    ratio = [ratio, 1 - ratio]
    splits = []
    prev_threshold = None
    for threshold in np.cumsum(ratio):
        condition = df["rank_s"] <= round(threshold * df["count"])
        if prev_threshold is not None:
            condition &= df["rank_s"] > round(prev_threshold * df["count"])
        splits.append(df[condition].drop(["rank_s", "count"], axis=1))
        prev_threshold = threshold

    return splits

def train_val_split(
        train_df: pd.DataFrame, 
        val_users_n: int = 200_000):
    
    user_ids = train_df['user_id'].unique()
    user_ids_val = random.sample(list(user_ids), val_users_n)
    condition = train_df['user_id'].isin(user_ids_val)

    val = train_df[condition]
    val_no_targets, val_targets = chrono_split(val, ratio=0.7)

    train = pd.concat([train_df[~condition], val_no_targets]).sort_values('timestamp')
    return train, val_no_targets, val_targets

In [5]:
random.seed(56)
train, val_no_targets, val_targets = train_val_split(train_df,val_users_n=100_000)

In [6]:
idxes = train.index

In [7]:
train['user_id'].iloc[0]

40821

In [8]:
train['video_id'].value_counts()[:10]

video_id
166825    17901
131005    17146
38943     16410
188589    15830
169519    15394
34204     15272
51799     12325
137447    12091
166282    11737
123953    10883
Name: count, dtype: int64

In [9]:
val_no_targets[val_no_targets['interaction_type'] != 'view'].shape[0] / val_no_targets.shape[0]

0.17421546088274967

In [10]:
def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [11]:
val_labels = val_targets.groupby('user_id')['video_id'].apply(lambda x:x.tolist()).tolist()

In [12]:
### BaseLine_Simple

train_best_cols = train['video_id'].value_counts().index[:10]
mapk(val_labels,[train_best_cols] * len(val_labels),k=10)

0.006894914600655077

In [13]:
train['is_like'] = train['interaction_type'].apply(lambda x: 1 if x == 'like' else 0)
video_features = train.groupby('video_id')['is_like'].agg(['sum','count'])

In [14]:
video_features['index'] = video_features['sum'] * 10 + video_features['count']

In [15]:
video_features['video_id'] = video_features.index

In [16]:
train_best_cols = video_features.sort_values(by='index')[::-1][:10].index.tolist()
mapk(val_labels,[train_best_cols] * len(val_labels),k=10)

0.007281970669564121

In [17]:
#### BM25
video_features[['video_id','index']].index = range(video_features.shape[0])
video_features['range'] = range(video_features.shape[0])
video_features = video_features.set_index('range')

In [18]:
#train = train.drop('index',axis=1)
train = train.merge(video_features[['video_id','index']],on='video_id')

In [19]:
train['index'] = train['index'].map(lambda x: x ** 0.25)

In [20]:
users_inv_mapping = dict(enumerate(train_df['user_id'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}

items_inv_mapping = dict(enumerate(train_df['video_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}
len(users_mapping),len(items_mapping)

(152911, 228506)

In [21]:
def get_coo_matrix(df,
                   user_col='user_id',
                   item_col='item_id',
                   weight_col=None,
                   users_mapping=None,
                   items_mapping=None):
    if weight_col is None:
        weights = np.ones(len(df), dtype=np.float32)
    else:
        weights = df[weight_col].astype(np.float32)

    interaction_matrix = sp.coo_matrix((
        weights,
        (
            df[user_col].map(users_mapping.get),
            df[item_col].map(items_mapping.get)
        )),
    )
    return interaction_matrix

In [27]:
train_mat = get_coo_matrix(df=train,
                           user_col='user_id',
                           item_col='video_id',
                           weight_col='index', 
                           users_mapping=users_mapping,
                           items_mapping=items_mapping).tocsr()

In [28]:
#model = AlternatingLeastSquares(
#    factors=32, iterations=128,
#    random_state=56
#)
#model = BayesianPersonalizedRanking(factors=32, iterations=16,random_state=56)
#model = LogisticMatrixFactorization(factors=32, iterations=50,random_state=56)
model = TFIDFRecommender(K=100)
#model = BM25Recommender()

model.fit(train_mat)

c:\Users\kostopr4v\AppData\Local\Programs\Python\Python310\lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.02299976348876953 seconds
  warnings.warn(
100%|██████████| 228506/228506 [00:01<00:00, 195404.09it/s]


In [22]:
def predict_impl(model,test_users,mat,users_mapping,items_inv_mapping,N=10,falh=True):
    recs,scores = [],[]
    for id in tqdm(test_users):
        row_id = users_mapping[id]
        ranks = model.recommend(row_id, mat[row_id], N=N, filter_already_liked_items=falh)
        recs += [[items_inv_mapping.get(it) for it in ranks[0]]]
        scores += [ranks[1]]
    return recs,scores

def predict_impl_batched(model,test_users,mat,users_mapping,items_inv_mapping,batch_size=1024,N=10,falh=True):
    recs,scores = [],[]
    N = len(test_users)
    for i in tqdm(range(0,N,batch_size)):
        ids = test_users[i:i+batch_size]
        row_id = [users_mapping[id] for id in ids]
        ranks_lst = model.recommend(row_id, mat[row_id], N=N, filter_already_liked_items=falh)
        for ranks in ranks_lst:
            recs += [[items_inv_mapping.get(it) for it in ranks[0]]]
            scores += [ranks[1]]
    return recs,scores

In [30]:
val_group = val_targets.groupby('user_id')
act = val_group['video_id'].agg(lambda x:x.tolist()).tolist()
val_users = val_group.agg(lambda x:x.tolist()).index.tolist()

In [31]:
test_preds,test_scores = predict_impl(model,
                                      val_users,
                                      train_mat,
                                      users_mapping,
                                      items_inv_mapping,
                                      N=100,
                                      falh=True)

100%|██████████| 100000/100000 [00:13<00:00, 7263.62it/s]


In [32]:
mapk(val_labels,test_preds,k=100)

0.02945191225524994

In [33]:
pd.DataFrame({'preds':test_preds,'scores':test_scores}).to_parquet('TFIDF_preds_val100.parquet')

In [23]:
train_df['is_like'] = train_df['interaction_type'].apply(lambda x: 1 if x == 'like' else 0)
video_features = train_df.groupby('video_id')['is_like'].agg(['sum','count'])
video_features['index'] = video_features['sum'] * 10 + video_features['count']
video_features['video_id'] = video_features.index
video_features[['video_id','index']].index = range(video_features.shape[0])
video_features['range'] = range(video_features.shape[0])
video_features = video_features.set_index('range')
train_df = train_df.merge(video_features[['video_id','index']],on='video_id')
train_df['index'] = train_df['index'].map(lambda x: x ** 0.25)

In [24]:
train_mat

NameError: name 'train_mat' is not defined

In [25]:
train_mat = get_coo_matrix(df=train_df,
                           user_col='user_id',
                           item_col='video_id',
                           weight_col='index',
                           users_mapping=users_mapping,
                           items_mapping=items_mapping).tocsr()

In [26]:
model = TFIDFRecommender(K=100)

model.fit(train_mat)

c:\Users\kostopr4v\AppData\Local\Programs\Python\Python310\lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.02755141258239746 seconds
  warnings.warn(
100%|██████████| 228506/228506 [00:01<00:00, 140384.66it/s]


In [27]:
sample_sub = pd.read_csv('lol.csv')
test_id = sample_sub['user_id'].tolist()

In [28]:
test_preds,test_scores = predict_impl(model,
                                      test_id,
                                      train_mat,
                                      users_mapping,
                                      items_inv_mapping,
                                      N=100,
                                      falh=True)

100%|██████████| 152911/152911 [00:27<00:00, 5662.37it/s]


In [29]:
pd.DataFrame({'preds':test_preds,'scores':test_scores}).to_parquet('TFIDF_preds_test100.parquet')

In [30]:
predicted_cols=[' '.join(map(str, i)) for i in test_preds]
sample_sub['recommendation'] = predicted_cols

In [31]:
sample_sub['count_tr'] = sample_sub['recommendation'].map(lambda x: len(x.split()))

In [32]:
sample_sub[sample_sub['count_tr'] != 10]

user_id                                     recommendation  count_tr
0           938  6 18999 14803 72256 127197 102474 207669 12422...       100
1         57571  34204 169519 188589 166282 137447 54812 214736...       100
2         50873  37487 3402 200362 70460 52442 248427 148191 15...       100
3          4335  131005 188589 34204 169519 133062 166282 13009...       100
4         42138  230006 73271 51799 20630 170373 193601 131005 ...       100
...         ...                                                ...       ...
152906    79591  117763 230387 108084 10748 58865 62430 22914 1...       100
152907    93648  89801 224385 149695 25243 115474 118575 193340...       100
152908    42337  166825 169519 245475 131005 34204 54812 207669...       100
152909     4694  248488 224218 153743 112291 7216 208480 9054 4...       100
152910    25414  13009 89274 133062 169519 166282 34204 131005 ...       100

[152911 rows x 3 columns]

In [33]:
recs = sample_sub['recommendation']
recs[98527] = '24671 65611 45311 132648 217384 34204 166282 51799 131005 188589'
recs[98927] = '24671 65611 45311 132648 217384 34204 166282 51799 131005 188589'
recs[144672] = '24671 65611 45311 132648 217384 34204 166282 51799 131005 188589'

C:\Users\kostopr4v\AppData\Local\Temp\ipykernel_10304\3122395054.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recs[98527] = '24671 65611 45311 132648 217384 34204 166282 51799 131005 188589'
C:\Users\kostopr4v\AppData\Local\Temp\ipykernel_10304\3122395054.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recs[98927] = '24671 65611 45311 132648 217384 34204 166282 51799 131005 188589'
C:\Users\kostopr4v\AppData\Local\Temp\ipykernel_10304\3122395054.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan

In [34]:
sample_sub[['recommendation','user_id']]

recommendation  user_id
0       6 18999 14803 72256 127197 102474 207669 12422...      938
1       34204 169519 188589 166282 137447 54812 214736...    57571
2       37487 3402 200362 70460 52442 248427 148191 15...    50873
3       131005 188589 34204 169519 133062 166282 13009...     4335
4       230006 73271 51799 20630 170373 193601 131005 ...    42138
...                                                   ...      ...
152906  117763 230387 108084 10748 58865 62430 22914 1...    79591
152907  89801 224385 149695 25243 115474 118575 193340...    93648
152908  166825 169519 245475 131005 34204 54812 207669...    42337
152909  248488 224218 153743 112291 7216 208480 9054 4...     4694
152910  13009 89274 133062 169519 166282 34204 131005 ...    25414

[152911 rows x 2 columns]

In [35]:
sample_sub['recommendation'] = recs

In [36]:
sample_sub[['recommendation','user_id']].to_csv('TF_IDF_REWEIGHTV2_025.csv',index=False)

конец


In [3]:
%pip install fastparquet pyarrow

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\kostopr4v\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
sample_sub

In [4]:
import pandas as pd
a = pd.read_parquet("TFIDF_preds_test100.parquet")


In [5]:
a.head()

preds  \
0  [6, 18999, 14803, 72256, 127197, 102474, 20766...   
1  [34204, 169519, 188589, 166282, 137447, 54812,...   
2  [37487, 3402, 200362, 70460, 52442, 248427, 14...   
3  [131005, 188589, 34204, 169519, 133062, 166282...   
4  [230006, 73271, 51799, 20630, 170373, 193601, ...   

                                              scores  
0  [6.4325948788303755, 6.0271326543031005, 5.720...  
1  [22.697123564241036, 21.293344826673533, 20.13...  
2  [25.46751850863799, 25.087862314484468, 23.997...  
3  [11.830583708850314, 11.325816584084441, 11.26...  
4  [13.771759742893257, 12.926215190266484, 10.85...

In [9]:
b = pd.read_csv('150_candidates.csv')
b

user_id                                           video_id  \
0        126492  [ 13009  69926  89274 211256 221242 102474 121...   
1        117764  [ 53929  28018  63158  10081 166381  12464 199...   
2         11347  [ 26890   6551 212556 152078  18999 102474    ...   
3        125274  [207669  38943 132648  77671  45326 123953 201...   
4         67028  [ 63190 148139  93961 171741 133062  60830  17...   
...         ...                                                ...   
152906   122568  [214736 131005  45311 124395  54812  64957 103...   
152907   143111  [207669 201397  45326 245475  43931 193340  49...   
152908    37071  [ 63190  17689 109506  34935  93961 106613 141...   
152909     3642  [ 62222  95381 101067 152078 170313 133923 212...   
152910   146066  [161789 174786 111527  32275  35224 233131  99...   

                                                    score  
0       [0.31930417 0.30017498 0.17347696 0.14865126 0...  
1       [0.28012246 0.2589475  0.2547775  0.23732443 0...  
2       [0.56107867 0.34114453 0.31112826 0.2661739  0...  
3       [0.00795556 0.00735591 0.00635    0.00579679 0...  
4       [0.0614441  0.05303272 0.05296801 0.05030395 0...  
...                                                   ...  
152906  [0.06514145 0.04484507 0.04072596 0.04061606 0...  
152907  [0.01590715 0.01361683 0.0095805  0.00937266 0...  
152908  [0.0895606  0.07612623 0.07464532 0.0629826  0...  
152909  [0.0303891  0.01863679 0.01772885 0.01679858 0...  
152910  [0.02625357 0.02231028 0.02030385 0.0197309  0...  

[152911 rows x 3 columns]

In [13]:
b['video_id'][0].replace("\n", "").replace("[", "").replace("]", "").split()

['13009',
 '69926',
 '89274',
 '211256',
 '221242',
 '102474',
 '121660',
 '33725',
 '45223',
 '173322',
 '51358',
 '96111',
 '6',
 '98235',
 '172208',
 '144291',
 '78554',
 '18999',
 '189727',
 '233363',
 '14803',
 '207924',
 '117032',
 '111331',
 '229320',
 '179178',
 '45664',
 '127197',
 '60830',
 '214143',
 '126229',
 '17689',
 '64957',
 '171741',
 '124769',
 '78335',
 '133062',
 '156058',
 '17183',
 '103565',
 '65611',
 '122373',
 '132060',
 '159395',
 '65941',
 '222069',
 '230424',
 '163989',
 '78578',
 '191191',
 '43275',
 '245249',
 '170313',
 '26721',
 '101067',
 '81779',
 '28340',
 '148896',
 '124224',
 '139928',
 '118239',
 '139205',
 '24705',
 '208511',
 '179405',
 '166381',
 '226671',
 '222733',
 '190931',
 '54812',
 '239932',
 '95381',
 '240523',
 '227191',
 '165997',
 '122759',
 '153488',
 '126534',
 '11250',
 '174544',
 '159202',
 '228318',
 '233087',
 '207447',
 '69651',
 '12926',
 '96522',
 '201337',
 '62544',
 '179468',
 '706',
 '237329',
 '54200',
 '84145',
 '93961'

In [14]:
def kek(x):
    return x.replace("\n", "").replace("[", "").replace("]", "").split()

b['video_id'] = b['video_id'].apply(kek)

In [16]:
b['score'] = b['score'].apply(kek)

In [19]:
b = b.set_index("user_id")
b.index.name = None

In [20]:
b

video_id  \
126492  [13009, 69926, 89274, 211256, 221242, 102474, ...   
117764  [53929, 28018, 63158, 10081, 166381, 12464, 19...   
11347   [26890, 6551, 212556, 152078, 18999, 102474, 6...   
125274  [207669, 38943, 132648, 77671, 45326, 123953, ...   
67028   [63190, 148139, 93961, 171741, 133062, 60830, ...   
...                                                   ...   
122568  [214736, 131005, 45311, 124395, 54812, 64957, ...   
143111  [207669, 201397, 45326, 245475, 43931, 193340,...   
37071   [63190, 17689, 109506, 34935, 93961, 106613, 1...   
3642    [62222, 95381, 101067, 152078, 170313, 133923,...   
146066  [161789, 174786, 111527, 32275, 35224, 233131,...   

                                                    score  
126492  [0.31930417, 0.30017498, 0.17347696, 0.1486512...  
117764  [0.28012246, 0.2589475, 0.2547775, 0.23732443,...  
11347   [0.56107867, 0.34114453, 0.31112826, 0.2661739...  
125274  [0.00795556, 0.00735591, 0.00635, 0.00579679, ...  
67028   [0.0614441, 0.05303272, 0.05296801, 0.05030395...  
...                                                   ...  
122568  [0.06514145, 0.04484507, 0.04072596, 0.0406160...  
143111  [0.01590715, 0.01361683, 0.0095805, 0.00937266...  
37071   [0.0895606, 0.07612623, 0.07464532, 0.0629826,...  
3642    [0.0303891, 0.01863679, 0.01772885, 0.01679858...  
146066  [0.02625357, 0.02231028, 0.02030385, 0.0197309...  

[152911 rows x 2 columns]

In [21]:
b.to_csv('als_tfidf.csv')

In [6]:
from sklearn.model_selection import train_test_split

global_train, global_test = train_test_split(train_df, test_size=0.2)

In [7]:
global_train

interaction_type      timestamp  user_id  video_id
4179726             view  1697613486788    46584    133062
856108              like  1697565693256    51556    154082
294757              view  1696701274592   120534    114432
4644754             view  1696237765848    61554    193494
2301974             view  1696529293697   111218    123953
...                  ...            ...      ...       ...
1485461             view  1696987415820   111190    103595
2153484             view  1697546109405    37176    234888
5080656             view  1696815256153    12105    211730
996739              view  1697745009225    33459    127368
3882045             view  1698572569512   134234     23761

[4525859 rows x 4 columns]

In [8]:
# init class
from lightfm.data import Dataset
dataset = Dataset()

# fit tuple of user and movie interactions
dataset.fit(global_train['user_id'].unique(), global_train['item_id'].unique())

ModuleNotFoundError: No module named 'lightfm'